In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import itertools
from matplotlib.colors import Normalize, LogNorm

sys.path.append("..")

In [ ]:
from manage_data import get_conditions, load_result, delete_all, delete_result

In [ ]:
# delete_all()

In [ ]:
df = load_result(conditions=get_conditions(
    N_particle = None,
    N_ensemble = None,
    velocity = None,
    Lambda = None,
    boundary = None,
    N_bins = None,
    gamma = None,
    slope = None,
    temperature = None,
    tau = None,
    Da = None,
    delta_t = None,
    initial = None,
    sampling = None
    )
)

df

In [ ]:
# delete = df[df["size"] == 128]
delete = df
print(list(delete["key"]))
print(len(list(delete["key"])))
# delete_result(list(delete["key"]))

In [ ]:
x, y = sorted(set(df["slope"].to_numpy())), sorted(set(df["Lambda"].to_numpy()))
print(f"{x=}, {y=}")

In [ ]:
average, std, sample_size = (
    np.zeros((len(x), len(y))), np.zeros((len(x), len(y))), np.zeros((len(x), len(y))))

for i, slope in enumerate(x):
    for j, Lambda in enumerate(y):
        conditions = get_conditions(slope=slope, Lambda=Lambda)
        filtered_df = df.query(" and ".join(conditions))
        
        ensembles, N_ensembles = filtered_df["average"].to_numpy(), filtered_df["N_ensemble"].to_numpy()
        
        if len(ensembles) == 0: continue
        else:
            average[i,j] = np.mean(ensembles[0])
            std[i,j] = np.std(ensembles[0])
            sample_size[i,j] = N_ensembles[0]

mask = (sample_size == 0)
print(mask.astype(np.int64).sum())

average = np.transpose(average)
std = np.transpose(std)
sample_size = np.transpose(sample_size)

print(f"{average=} {std=} {sample_size=}")

In [ ]:
fig, ax = plt.subplots()

c = ax.pcolor(x, y, average, shading="auto",
               norm=Normalize(vmin=average.min(), vmax=0), cmap="bone")

fig.colorbar(c, ax=ax)

ax.set_title("AOUP drag")
ax.set_xlabel(rf"Force $f$")
ax.set_ylabel(rf"Size $\lambda$")

plt.show()

In [ ]:
from scipy.stats import norm

Z = (average - 0) / (std / np.sqrt(sample_size))
Z[np.isnan(Z)] = 0.0

p_value = norm.cdf(Z)
print(p_value.min())

fig, ax = plt.subplots()

c = ax.pcolor(x, y, p_value, shading='auto',
               norm=LogNorm(vmin=p_value.min(), vmax=1.e-2), cmap="bone")

fig.colorbar(c, ax=ax)

ax.set_title("T-Test p-value")
ax.set_xlabel(rf"Force $f$")
ax.set_ylabel(rf"Size $\lambda$")

plt.show()

In [ ]:
import numpy as np
import numpy.typing as npt
from numba import njit

@njit
def mask(arr: npt.NDArray, mask: npt.NDArray):
    force = np.zeros(shape=(10, 10)).reshape(-1)
    mask2 = force > 1
    force[mask2] = 0
    
    return force.reshape(10,10)

print(mask(np.array([1,2,3]), np.array([True, False, True])))

In [17]:
import numpy as np
Lambdas = np.round(np.linspace(0.01, 0.25, 25), 2)
print(Lambdas)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25]
